In [1]:
import pandas as pd
import pd_explain

usetex-False
(CVXPY) Jan 21 09:48:25 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jan 21 09:48:25 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.11.4210). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
adults = pd.read_csv(r"..\Examples\Datasets\adult.csv")

# Using the many to one explainer on the entire dataset:

Using the many to one explainer on a column from the dataset that has a many to one relationship with other columns (the visualization uses PCA for dimensionality reduction):

In [3]:
adults.explain(explainer='many_to_one', labels='label', coverage_threshold=0.8, separation_threshold=0.2)

Coverage Separation Error
Group / Cluster Explanation                                          
<=50K           0 <= capital-gain <= 4668.5     0.99              0.2
                0 <= capital-gain <= 4243.5     0.99              0.2
>50K            No explanation found             NaN              NaN

# Using the many to one explainer on a subset of the dataset:

Using the many to one explainer on subsets of the data, that only contain numerical / categorical columns:

In [4]:
adults_numeric_only = adults[['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']]
adults_numeric_only

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40
...,...,...,...,...,...,...
48837,39,215419,13,0,0,36
48838,64,321403,9,0,0,40
48839,38,374983,13,0,0,50
48840,44,83891,13,5455,0,40


Just like we could provide a column name (if it is a column in the dataset), we can also pass a pandas series as the labels parameter:

In [5]:
labels = adults['label']

In [6]:
adults_numeric_only.explain(explainer='many_to_one', labels=labels, coverage_threshold=0.8, separation_threshold=0.2)

Coverage Separation Error
Group / Cluster Explanation                                          
<=50K           0 <= capital-gain <= 4668.5     0.99              0.2
                0 <= capital-gain <= 4243.5     0.99              0.2
>50K            No explanation found             NaN              NaN

In [7]:
adults_categorical_only = adults[['workclass', 'education', 'marital-status', 'occupation', 'relationship']]
adults_categorical_only

,workclass,education,marital-status,occupation,relationship
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife
...,...,...,...,...,...
48837,Private,Bachelors,Divorced,Prof-specialty,Not-in-family
48838,?,HS-grad,Widowed,?,Other-relative
48839,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband
48840,Private,Bachelors,Divorced,Adm-clerical,Own-child


In [8]:
adults_categorical_only.explain(explainer='many_to_one', labels=labels, coverage_threshold=0.8, separation_threshold=0.3)

Coverage  \
Group / Cluster Explanation                                                   
<=50K           occupation != Prof-specialty                           0.91   
                occupation != Prof-specialty AND education != B...     0.82   
>50K            No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
<=50K           occupation != Prof-specialty                                   0.21  
                occupation != Prof-specialty AND education != B...             0.18  
>50K            No explanation found                                            NaN

We can also specify only specific columns to be used in the many to one explainer:

In [9]:
adults.explain(explainer='many_to_one', attributes=['age', 'education', 'marital-status', 'relationship'], labels='label', coverage_threshold=0.8, separation_threshold=0.5)

,,Coverage,Separation Error
Group / Cluster,Explanation,,
<=50K,education != Bachelors,0.87,0.21
>50K,No explanation found,NaN,NaN


# Using many to one explainer with clustering labels:

In [10]:
from sklearn.cluster import KMeans

In [11]:
adults_with_dummies = pd.get_dummies(adults)

In [12]:
cluster_labels = KMeans(n_clusters=3).fit_predict(adults_with_dummies)

C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Yuval\PycharmProjects\FEDEx_Generator\.venv1\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Yuval\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  Fi

In [13]:
adults.explain(explainer='many_to_one', labels=cluster_labels, coverage_threshold=0.8, separation_threshold=0.2)

Coverage  \
Group / Cluster Explanation                                                   
0               70654.5 <= capital-gain <= 99999 AND 12285.0 <=...      1.0   
                83332.5 <= capital-gain <= 99999.0 AND 12285.0 ...      1.0   
                74999.25 <= capital-gain <= 99999.0 AND 12285.0...      1.0   
                49999.5 <= capital-gain <= 99999.0 AND 12285.0 ...     0.99   
                66666.0 <= capital-gain <= 99999.0 AND 12285.0 ...     0.99   
                79999.2 <= capital-gain <= 99999.0 AND 12285.0 ...      1.0   
                capital-gain == 99999 AND 12285.0 <= fnlwgt <= ...     0.99   
                83332.5 <= capital-gain <= 99999.0 AND 12285.0 ...     0.99   
                74999.25 <= capital-gain <= 99999.0 AND 12285.0...     0.99   
                66666.0 <= capital-gain <= 99999.0 AND 12285.0 ...      1.0   
                70654.5 <= capital-gain <= 99999 AND 12285.0 <=...     0.99   
                79999.2 <= capital-gain <= 99999.0 AND 12285.0 ...     0.99   
                49999.5 <= capital-gain <= 99999.0                      1.0   
                capital-gain == 99999                                   1.0   
                66666.0 <= capital-gain <= 99999.0                      1.0   
                capital-gain == 99999 AND 12285.0 <= fnlwgt <= ...      1.0   
                83332.5 <= capital-gain <= 99999.0                      1.0   
                49999.5 <= capital-gain <= 99999.0 AND 12285.0 ...      1.0   
                79999.2 <= capital-gain <= 99999.0                      1.0   
                70654.5 <= capital-gain <= 99999                        1.0   
                74999.25 <= capital-gain <= 99999.0                     1.0   
1               0 <= capital-gain <= 30961.5 AND 12285 <= fnlwg...      1.0   
                12285 <= fnlwgt <= 234204.0                             1.0   
                0.0 <= capital-gain <= 33333.0 AND 12285 <= fnl...      1.0   
                0.0 <= capital-gain <= 16666.5 AND 12285 <= fnl...      1.0   
                0.0 <= capital-gain <= 19999.8 AND 12285 <= fnl...      1.0   
                0.0 <= capital-gain <= 49999.5 AND 12285 <= fnl...      1.0   
                0.0 <= capital-gain <= 24999.75 AND 12285 <= fn...      1.0   
2               234216.5 <= fnlwgt <= 1490400                           1.0   
                234204.0 <= fnlwgt <= 1490400                           1.0   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
0               70654.5 <= capital-gain <= 99999 AND 12285.0 <=...             0.05  
                83332.5 <= capital-gain <= 99999.0 AND 12285.0 ...             0.05  
                74999.25 <= capital-gain <= 99999.0 AND 12285.0...             0.05  
                49999.5 <= capital-gain <= 99999.0 AND 12285.0 ...              0.0  
                66666.0 <= capital-gain <= 99999.0 AND 12285.0 ...              0.0  
                79999.2 <= capital-gain <= 99999.0 AND 12285.0 ...             0.05  
                capital-gain == 99999 AND 12285.0 <= fnlwgt <= ...              0.0  
                83332.5 <= capital-gain <= 99999.0 AND 12285.0 ...              0.0  
                74999.25 <= capital-gain <= 99999.0 AND 12285.0...              0.0  
                66666.0 <= capital-gain <= 99999.0 AND 12285.0 ...             0.05  
                70654.5 <= capital-gain <= 99999 AND 12285.0 <=...              0.0  
                79999.2 <= capital-gain <= 99999.0 AND 12285.0 ...              0.0  
                49999.5 <= capital-gain <= 99999.0                             0.18  
                capital-gain == 99999                                          0.18  
                66666.0 <= capital-gain <= 99999.0                             0.18  
                capital-gain == 99999 AND 12285.0 <= fnlwgt <= ...          

# Using many to one explainer to explain groupby groups:

Many to one explainer can also be used to explain groupby groups. It can work with single or multiple attributes groupby:

In [14]:
single_attribute_groupby = adults.groupby('relationship').mean()
single_attribute_groupby

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
relationship,,,,,,
Husband,43.906066,187193.324102,10.305133,1774.499797,122.200649,44.166109
Not-in-family,38.421601,190352.245013,10.326870,783.963284,76.840817,40.529842
Other-relative,33.420983,203524.602258,8.849270,234.989376,49.920983,37.128154
Own-child,24.794090,193782.299697,9.494394,174.381480,41.276217,33.142461
Unmarried,40.312195,191370.284878,9.631610,450.863805,46.638634,39.172488
Wife,40.078507,180748.781639,10.488631,1658.779923,116.033033,36.729730


To explain a groupby's groups, we can use the explainer on the groupby dataframe without specifying the labels parameter:

In [15]:
single_attribute_groupby.explain(explainer='many_to_one', coverage_threshold=0.8, separation_threshold=0.2)

Coverage  \
Group / Cluster Explanation                                                   
Husband         marital-status == Married-civ-spouse AND sex !=...      1.0   
                marital-status == Married-civ-spouse                    1.0   
                marital-status == Married-civ-spouse AND sex ==...      1.0   
Wife            marital-status == Married-civ-spouse AND sex !=...     0.99   
                marital-status == Married-civ-spouse AND sex ==...     0.99   
Not-in-family   No explanation found                                    NaN   
Other-relative  No explanation found                                    NaN   
Own-child       No explanation found                                    NaN   
Unmarried       No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Husband         marital-status == Married-civ-spouse AND sex !=...             0.01  
                marital-status == Married-civ-spouse                           0.12  
                marital-status == Married-civ-spouse AND sex ==...             0.01  
Wife            marital-status == Married-civ-spouse AND sex !=...             0.07  
                marital-status == Married-civ-spouse AND sex ==...             0.07  
Not-in-family   No explanation found                                            NaN  
Other-relative  No explanation found                                            NaN  
Own-child       No explanation found                                            NaN  
Unmarried       No explanation found                                            NaN

In [16]:
multi_attribute_groupby = adults.groupby(['sex', 'label']).mean()
multi_attribute_groupby

age         fnlwgt  education-num  capital-gain  \
sex    label                                                          
Female <=50K  36.278999  185690.282604       9.824724    122.876586   
       >50K   42.219333  183989.519503      11.832109   4313.663652   
Male   <=50K  37.248548  192799.098671       9.454953    162.322673   
       >50K   44.641863  189269.827082      11.561908   3993.827586   

              capital-loss  hours-per-week  
sex    label                                
Female <=50K     46.910976       35.875962  
       >50K     180.222725       40.678915  
Male   <=50K     58.746173       40.720702  
       >50K     195.902299       46.304396

In [17]:
multi_attribute_groupby.explain(explainer='many_to_one', coverage_threshold=0.8, separation_threshold=0.55)

Coverage  \
Group / Cluster     Explanation                                                   
('Male', '<=50K')   0 <= capital-gain <= 99999                              1.0   
                    0 <= capital-gain <= 4668.5                            0.99   
                    0.0 <= capital-gain <= 19999.8                          1.0   
                    0.0 <= capital-gain <= 33333.0                          1.0   
                    relationship != infrequent_sklearn AND 0 <= cap...     0.96   
                    0.0 <= capital-gain <= 24999.75                         1.0   
                    0 <= capital-gain <= 4932.5                            0.99   
                    0.0 <= capital-gain <= 16666.5                          1.0   
                    0.0 <= capital-gain <= 49999.5                          1.0   
                    relationship != infrequent_sklearn AND 0 <= cap...     0.96   
('Female', '<=50K') relationship != Husband AND 0 <= capital-gain <...     0.99   
                    relationship != Husband                                 1.0   
                    relationship != Husband AND 0 <= capital-gain <...     0.99   
('Male', '>50K')    relationship == Husband                                0.89   
('Female', '>50K')  No explanation found                                    NaN   

                                                                       Separation Error  
Group / Cluster     Explanation                                                          
('Male', '<=50K')   0 <= capital-gain <= 99999                                     0.53  
                    0 <= capital-gain <= 4668.5                                    0.51  
                    0.0 <= capital-gain <= 19999.8                                 0.53  
                    0.0 <= capital-gain <= 33333.0                                 0.53  
                    relationship != infrequent_sklearn AND 0 <= cap...             0.49  
                    0.0 <= capital-gain <= 24999.75                                0.53  
                    0 <= capital-gain <= 4932.5                                    0.51  
                    0.0 <= capital-gain <= 16666.5                                 0.53  
                    0.0 <= capital-gain <= 49999.5                                 0.53  
                    relationship != infrequent_sklearn AND 0 <= cap...             0.49  
('Female', '<=50K') relationship != Husband AND 0 <= capital-gain <...             0.49  
                    relationship != Husband                                         0.5  
                    relationship != Husband AND 0 <= capital-gain <...             0.49  
('Male', '>50K')    relationship == Husband                                        0.55  
('Female', '>50K')  No explanation found                                            NaN

It is also possible to pass a list of labels to the many to one explainer.
It will then treat that list as groupby groups and explain each group separately, as if a groupby was performed:

In [18]:
adults.explain(explainer='many_to_one', coverage_threshold=0.8, separation_threshold=0.55, labels=['sex', 'label'])

Coverage  \
Group / Cluster     Explanation                                                   
('Male', '<=50K')   0 <= capital-gain <= 99999                              1.0   
                    0 <= capital-gain <= 4668.5                            0.99   
                    0.0 <= capital-gain <= 19999.8                          1.0   
                    0.0 <= capital-gain <= 33333.0                          1.0   
                    relationship != infrequent_sklearn AND 0 <= cap...     0.96   
                    0.0 <= capital-gain <= 24999.75                         1.0   
                    0 <= capital-gain <= 4932.5                            0.99   
                    0.0 <= capital-gain <= 16666.5                          1.0   
                    0.0 <= capital-gain <= 49999.5                          1.0   
                    relationship != infrequent_sklearn AND 0 <= cap...     0.96   
('Female', '<=50K') relationship != Husband AND 0 <= capital-gain <...     0.99   
                    relationship != Husband                                 1.0   
                    relationship != Husband AND 0 <= capital-gain <...     0.99   
('Male', '>50K')    relationship == Husband                                0.89   
('Female', '>50K')  No explanation found                                    NaN   

                                                                       Separation Error  
Group / Cluster     Explanation                                                          
('Male', '<=50K')   0 <= capital-gain <= 99999                                     0.53  
                    0 <= capital-gain <= 4668.5                                    0.51  
                    0.0 <= capital-gain <= 19999.8                                 0.53  
                    0.0 <= capital-gain <= 33333.0                                 0.53  
                    relationship != infrequent_sklearn AND 0 <= cap...             0.49  
                    0.0 <= capital-gain <= 24999.75                                0.53  
                    0 <= capital-gain <= 4932.5                                    0.51  
                    0.0 <= capital-gain <= 16666.5                                 0.53  
                    0.0 <= capital-gain <= 49999.5                                 0.53  
                    relationship != infrequent_sklearn AND 0 <= cap...             0.49  
('Female', '<=50K') relationship != Husband AND 0 <= capital-gain <...             0.49  
                    relationship != Husband                                         0.5  
                    relationship != Husband AND 0 <= capital-gain <...             0.49  
('Male', '>50K')    relationship == Husband                                        0.55  
('Female', '>50K')  No explanation found                                            NaN

# Many to one explainer when there are many clusters:

In [19]:
from sklearn.cluster import KMeans
adults_with_dummies = pd.get_dummies(adults)

In [20]:
many_clusters = KMeans(n_clusters=11).fit_predict(adults_with_dummies)

In the case there are many clusters (over 10), the layout of the plot will be different:

In [21]:
adults.explain(explainer='many_to_one', labels=many_clusters, coverage_threshold=0.8, separation_threshold=0.2, explanation_form='disj')

,,Coverage,Separation Error
Group / Cluster,Explanation,,
0,(76394.0 <= fnlwgt <= 128943.0),1.0,0.0
4,(128943.0 <= fnlwgt <= 171074.5),1.0,0.0
8,(12285 <= fnlwgt <= 76394.0),1.0,0.0
9,(962971.0 <= fnlwgt <= 1490400),1.0,0.0
10,(639249.5 <= fnlwgt <= 1490400),1.0,0.13
1,No explanation found,NaN,NaN
2,No explanation found,NaN,NaN
3,No explanation found,NaN,NaN
5,No explanation found,NaN,NaN


# Disjunctive explanations:

The many-to-one explainer supports disjunctive explanation generation, as well as the conjunctive explanations shown above.
To generate disjunctive explanations, set the explanation_form parameter to 'disj' or 'disjunction':

In [28]:
single_attribute_groupby.explain(explainer='many_to_one', coverage_threshold=0.8, separation_threshold=0.2, explanation_form='disj')

Coverage  \
Group / Cluster Explanation                                                   
Husband         marital-status == Married-civ-spouse                    1.0   
                sex == Male OR marital-status == Married-civ-sp...      1.0   
                sex != Female OR marital-status == Married-civ-...      1.0   
Wife            sex == Female OR marital-status == Married-civ-...     0.99   
                sex != Male OR marital-status == Married-civ-sp...     0.99   
Not-in-family   No explanation found                                    NaN   
Other-relative  No explanation found                                    NaN   
Own-child       No explanation found                                    NaN   
Unmarried       No explanation found                                    NaN   

                                                                   Separation Error  
Group / Cluster Explanation                                                          
Husband         marital-status == Married-civ-spouse                           0.12  
                sex == Male OR marital-status == Married-civ-sp...             0.01  
                sex != Female OR marital-status == Married-civ-...             0.01  
Wife            sex == Female OR marital-status == Married-civ-...             0.07  
                sex != Male OR marital-status == Married-civ-sp...             0.07  
Not-in-family   No explanation found                                            NaN  
Other-relative  No explanation found                                            NaN  
Own-child       No explanation found                                            NaN  
Unmarried       No explanation found                                            NaN